<a href="https://colab.research.google.com/github/TeenaSingh4640/AshaAI/blob/main/Copy_of_final_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install python-docx PyPDF2 gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.4 MB/s eta 0:00:00


In [ ]:
import os
import sys
import json
import datetime
from openai import OpenAI
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
from rich.prompt import Prompt
from gtts import gTTS
from IPython.display import Audio, display
import PyPDF2
import docx
from urllib.parse import quote_plus
import spacy
from transformers import pipeline

# Initialize Rich Console
console = Console()

# OpenRouter client configuration
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

# Load bias detection model from Hugging Face
bias_detector = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# System prompt for Asha
SYSTEM_PROMPT = """
You are Asha, an AI assistant for the JobsForHer Foundation. Your purpose is to help women find jobs, mentorships,
events, and career resources. Always be supportive, inclusive, and empowering in your responses.

Your capabilities include:
- Providing job listings and career opportunities
- Suggesting relevant mentorship programs
- Sharing information about upcoming networking events
- Offering career advice and resources
- Answering questions about the job market for women
- When appropriate, include useful external links to official job portals, mentorship programs, or career resources (e.g., LinkedIn, JobsForHer, SHEROES, WomenWhoCode, etc.).
- If you cannot find an exact link, suggest popular trusted websites.
-Always output flowcharts in plain text ASCII art style using lines, boxes (+, -, |) and arrows (v), suitable for terminal display. Do NOT use Mermaid, Markdown code blocks, or graphical syntax.
-
Please maintain a professional but warm tone. If asked questions outside your scope or that contain bias,
politely redirect the conversation. Always prioritize privacy and never ask for or store personal information.

Current date: {current_date}
"""

class AshaBot:
    def __init__(self):
        self.conversation_history = []
        self.initialize_conversation()

    def initialize_conversation(self):
        current_date = datetime.datetime.now().strftime("%B %d, %Y")
        self.conversation_history = [
            {"role": "system", "content": SYSTEM_PROMPT.format(current_date=current_date)}
        ]

    def get_response(self, user_input):
        self.conversation_history.append({"role": "user", "content": user_input})

        try:
            with console.status("[bold green]Asha is thinking...", spinner="dots"):
                response = client.chat.completions.create(
                    model="google/gemini-2.5-flash-preview-04-17",
                    messages=self.conversation_history,
                    temperature=0.7,
                    max_tokens=1024,
                )

            ai_message = response.choices[0].message.content
            self.conversation_history.append({"role": "assistant", "content": ai_message})

            return ai_message

        except Exception as e:
            console.print(f"[bold red]Error communicating with AI service: {str(e)}[/bold red]")
            return "I'm sorry, I'm having trouble connecting to my services right now. Please try again later."

    def clear_conversation(self):
        self.initialize_conversation()
        return "Conversation history cleared. How can I assist you today?"

    def detect_bias(self, text):
        try:
            # Detect bias by using the Hugging Face model pipeline for zero-shot classification
            bias_labels = ["biased", "neutral", "inclusive"]
            result = bias_detector(text, candidate_labels=bias_labels)

            if result['labels'][0] == 'biased' and result['scores'][0] > 0.8:
                return True  # Bias detected
            return False  # No bias detected
        except Exception as e:
            console.print(f"[bold red]Error in bias detection: {e}[/bold red]")
            return False


def text_to_audio_and_play(text, lang):
    try:
        tts = gTTS(text=text, lang=lang)
        temp_filename = "temp_audio.mp3"
        tts.save(temp_filename)
        display(Audio(temp_filename, autoplay=True))
        os.remove(temp_filename)
    except Exception as e:
        console.print(f"[bold red]Audio playback failed: {e}[/bold red]")

def display_welcome_message():
    welcome_text = """
    # Welcome to Asha - Your AI Career Assistant

    Asha is here to help women advance their careers with:
    - Job opportunities tailored to your skills
    - Mentorship connections
    - Upcoming events and networking opportunities
    - Career resources and guidance

    Simply type your questions or requests, and Asha will assist you.
    Type `/help` to see available commands.
    """
    console.print(Panel(Markdown(welcome_text), border_style="green", title="JobsForHer Foundation", subtitle="Empowering Women's Careers"))

def display_help():
    help_text = """
    # Available Commands:

    - `/help` - Show this help message
    - `/clear` - Clear conversation history
    - `/exit` or `/quit` - Exit the chatbot
    - `/read <filepath>` - Upload and analyze a document

    For all other messages, Asha will provide a helpful response based on your query.
    """
    console.print(Panel(Markdown(help_text), border_style="blue", title="Help Commands"))

def read_document(file_path):
    text = ""
    try:
        if file_path.lower().endswith('.pdf'):
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    text += page.extract_text()
        elif file_path.lower().endswith('.docx'):
            doc = docx.Document(file_path)
            for para in doc.paragraphs:
                text += para.text + "\n"
        elif file_path.lower().endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
        else:
            return "Unsupported file type. Please upload a .pdf, .docx, or .txt file."
    except Exception as e:
        return f"Failed to read file: {e}"
    return text

def generate_direct_links(user_input):
    # Clean the input
    cleaned_input = user_input.lower()

    # Define base links for different categories
    base_links = {
        "jobs": {
            "LinkedIn": "https://www.linkedin.com/jobs/search/?keywords={query}",
            "Naukri": "https://www.naukri.com/{query}-jobs",
            "Indeed": "https://www.indeed.com/jobs?q={query}",
            "JobsForHer": "https://jobsforher.com/jobs/search?keywords={query}"
        },
        "mentorship": {
            "LinkedIn Mentorship": "https://www.linkedin.com/mentoring/",
            "MentorCruise": "https://mentorcruise.com/search/?q={query}",
            "JobsForHer Mentorship": "https://jobsforher.com/mentors"
        },
        "events": {
            "Eventbrite": "https://www.eventbrite.com/d/online/{query}/",
            "Meetup": "https://www.meetup.com/find/events/?allMeetups=true&keywords={query}",
            "JobsForHer Events": "https://jobsforher.com/events"
        }
    }

    # Decide the category
    category = "jobs"  # default
    if any(word in cleaned_input for word in ["mentor", "mentorship", "coaching", "guide"]):
        category = "mentorship"
    elif any(word in cleaned_input for word in ["event", "networking", "meetup", "conference", "workshop"]):
        category = "events"

    # Remove noise words for better query
    noise_words = ["find", "search", "looking for", "want", "interested in", "job", "jobs", "mentor", "mentorship", "event", "events", "meetup", "meetups", "program", "programs"]
    user_query = cleaned_input
    for word in noise_words:
        user_query = user_query.replace(word, "")
    user_query = user_query.strip()
    query_encoded = quote_plus(user_query if user_query else "career")

    # Generate links
    links = [f"[{site}]({url.format(query=query_encoded)})" for site, url in base_links[category].items()]

    return "Here are some links you can explore:\n\n" + "\n".join(links)

SUPPORTED_LANGUAGES = {
    'en': 'English',
    'hi': 'Hindi',
    'ta': 'Tamil',
    'es': 'Spanish',
    'fr': 'French',
    'de': 'German',
    'mr': 'Marathi',
    'te': 'Telugu',
}

def ask_preferred_language():
    console.print("\n[bold blue]Please select your preferred language:[/bold blue]")
    for code, lang in SUPPORTED_LANGUAGES.items():
        console.print(f"{code}: {lang}")
    chosen = Prompt.ask("\nEnter language code (default: en)", default="en")
    return chosen if chosen in SUPPORTED_LANGUAGES else "en"

def main():
    display_welcome_message()
    asha = AshaBot()
    user_language = ask_preferred_language()

    while True:
        user_input = Prompt.ask("\n[bold cyan]You[/bold cyan]")

        # Detect bias in user input
        if asha.detect_bias(user_input):
            console.print("\n[bold red]Warning: Your input may contain biased language. Please ensure your language is respectful and inclusive.[/bold red]")

        if user_input.lower() in ['/exit', '/quit']:
            console.print("[yellow]Thank you for using Asha. Goodbye![/yellow]")
            break
        elif user_input.lower() == '/help':
            display_help()
            continue
        elif user_input.lower() == '/clear':
            response = asha.clear_conversation()
            console.print(f"\n[bold purple]Asha[/bold purple]: {response}")
            continue
        elif user_input.lower().startswith('/read '):
            file_path = user_input[6:].strip()
            document_content = read_document(file_path)
            if document_content.startswith("Failed") or document_content.startswith("Unsupported"):
                console.print(f"\n[bold red]{document_content}[/bold red]")
                continue

            console.print(f"\n[bold blue]Document successfully read! Sending to Asha...[/bold blue]")
            ai_response = asha.get_response(
                f"I have uploaded a document. Please review it and suggest relevant jobs, mentorships, or events based on it:\n\n{document_content}"
            )
            console.print(f"\n[bold purple]Asha[/bold purple]: {ai_response}")
            text_to_audio_and_play(ai_response, user_language)
            continue

        ai_response = asha.get_response(user_input)
        external_links = generate_direct_links(user_input)

        full_response = ai_response
        if external_links:
            full_response += "\n\n" + external_links

        # Detect bias in AI response
        if asha.detect_bias(ai_response):
            console.print("\n[bold red]Warning: The AI response may contain biased language. Please review for inclusivity and neutrality.[/bold red]")

        console.print(Panel(Markdown(full_response), border_style="purple", title="Asha"))
        text_to_audio_and_play(full_response, user_language)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        console.print("\n[yellow]Program interrupted. Exiting...[/yellow]")
        sys.exit(0)

In [11]:
!pip install flask flask-cors pyngrok

# Add your authtoken
!ngrok config add-authtoken 2wIVrbl1vfbZiEY5coh5WxBpKEy_5t9jbUMF7bdVwhwVN9GBB

# Then start Flask + ngrok


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask flask-cors pyngrok

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import time

# Import or load your chatbot model here
# from your_trained_model import get_response

app = Flask(__name__)  # <<< FIXED HERE
CORS(app)  # Enable CORS for all routes

@app.route('/api/chat', methods=['POST'])
def chat():
    data = request.json
    user_message = data.get('message', '')

    if not user_message:
        return jsonify({'error': 'No message provided'}), 400

    try:
        # Replace with actual chatbot logic
        # response = get_response(user_message)

        # Temporary fake response
        response = f"This is a response to: {user_message}"

        return jsonify({
            'response': response,
            'status': 'success'
        })
    except Exception as e:
        return jsonify({
            'error': str(e),
            'status': 'error'
        }), 500

# Expose the Flask app using ngrok
ngrok_tunnel = ngrok.connect(5000)
print(f'Public URL: {ngrok_tunnel.public_url}')
print('Use this URL in your React app')

# Run Flask app in a separate thread
def run_app():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run_app).start()

# Keep the notebook alive
while True:
    time.sleep(60)
    print("Keeping Colab connection alive...")


Public URL: https://bae2-35-229-77-135.ngrok-free.app
Use this URL in your React app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 04:50:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 04:50:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 04:50:32] "GET / HTTP/1.1" 404 -


Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab connection alive...
Keeping Colab 